In [6]:
!pip install torch_geometric

In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import gym
import random
from collections import deque

# Load Dataset
df = pd.read_csv('/content/cleaned_toll_data.csv')

# Ensure proper data types
df = df.select_dtypes(include=[np.number]).dropna()

# ==========================
# 1. Random Forest for Queue Prediction
# ==========================
X = df.drop(columns=['queue_length'])  # Features
y = df['queue_length']  # Target

# Feature Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Hyperparameter-tuned Random Forest
rf = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print("Random Forest MAE:", mean_absolute_error(y_test, y_pred))
print("Random Forest RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

# ==========================
# 2. LSTM for Time-Series Queue Forecasting
# ==========================
time_series_data = df[['queue_length']].values
timesteps = 10

def create_sequences(data, timesteps):
    X, y = [], []
    for i in range(len(data) - timesteps):
        X.append(data[i:i+timesteps])
        y.append(data[i+timesteps])
    return np.array(X), np.array(y)

X, y = create_sequences(time_series_data, timesteps)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Improved LSTM Model
model = Sequential([
    LSTM(64, activation='relu', return_sequences=True, input_shape=(timesteps, 1)),
    Dropout(0.3),
    LSTM(64, activation='relu'),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

model.fit(X_train, y_train, epochs=30, batch_size=16, validation_data=(X_test, y_test))

# Evaluate LSTM
y_pred = model.predict(X_test)
print("LSTM MAE:", mean_absolute_error(y_test, y_pred))
print("LSTM RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

# ==========================
# 3. Deep Q-Learning for Toll Optimization
# ==========================
class TollEnv(gym.Env):
    def __init__(self):
        super(TollEnv, self).__init__()
        self.state = np.array([5, 10])  # [Queue Length, Toll Booths Open]
        self.action_space = gym.spaces.Discrete(3)  # 0: Open, 1: Close, 2: Maintain
        self.observation_space = gym.spaces.Box(low=0, high=100, shape=(2,), dtype=np.float32)

    def step(self, action):
        if action == 0:
            self.state[1] = min(self.state[1] + 1, 9)  # Open booth, max 9
        elif action == 1:
            self.state[1] = max(self.state[1] - 1, 0) # Close booth, min 0
        self.state[0] = max(0, self.state[0] - self.state[1])  # Reduce queue
        reward = -self.state[0]  # Less queue is better
        return self.state, reward, False, {}

    def reset(self):
        self.state = np.array([5, 10])
        return self.state

env = TollEnv()

# # Deep Q-Network (DQN) Model
# dqn_model = Sequential([
#     Dense(24, activation='relu', input_shape=(2,)),
#     Dense(24, activation='relu'),
#     Dense(env.action_space.n, activation='linear')
# ])
# dqn_model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))

# # Training DQN
# memory = deque(maxlen=2000)
# gamma, epsilon = 0.95, 1.0
# epsilon_min, epsilon_decay = 0.01, 0.995

# for episode in range(1000):
#     state = env.reset()
#     state = np.reshape(state, [1, 2])

#     for step in range(100):
#         # Epsilon-greedy action selection
#         if np.random.rand() <= epsilon:
#             action = env.action_space.sample()
#         else:
#             action = np.argmax(dqn_model.predict(state)[0])

#         next_state, reward, done, _ = env.step(action)
#         next_state = np.reshape(next_state, [1, 2])

#         # Store experience
#         memory.append((state, action, reward, next_state))

#         state = next_state

#     # Train DQN using replay memory
#     if len(memory) > 32:
#         minibatch = random.sample(memory, 32)
#         for state, action, reward, next_state in minibatch:
#             target = reward + gamma * np.amax(dqn_model.predict(next_state)[0])
#             target_f = dqn_model.predict(state)
#             target_f[0][action] = target
#             dqn_model.fit(state, target_f, epochs=1, verbose=0)

#     # Reduce exploration
#     if epsilon > epsilon_min:
#         epsilon *= epsilon_decay

# print("DQN Training Complete!")



Random Forest MAE: 2.390249076524137
Random Forest RMSE: 2.9511907268763444
Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - loss: 27.3801 - val_loss: 12.7182
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 12.5528 - val_loss: 10.7808
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 10.8185 - val_loss: 10.3431
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 11.4146 - val_loss: 9.7547
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 9.9236 - val_loss: 9.5411
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 9.4513 - val_loss: 9.6276
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 9.4220 - val_loss: 9.3238
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 10.1721 - val_loss: 9.3656
Epoch 9/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 10.3755 - val_loss: 9.2902
Epoch 10/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 9.6258 - val_loss: 9.6075
Epoch 11/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 9.3716 - val_loss: 9.5794
Epoch 12/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 9.366